In [1]:
from tkinter import *
from tkinter import ttk
from tkinter.ttk import *
from tkinter import filedialog,messagebox
import matplotlib.pyplot as plt
import os
import matplotlib
import parameters
import rnnfile
from keras import Sequential
from keras.layers import Dense, Conv1D, Input, MaxPooling1D, AveragePooling1D, Flatten, Dropout, LSTM
from osgeo import gdal
from tkinter.scrolledtext import ScrolledText

class PARAM:
    
    def __init__(self, master):
        self.valuesdata1 = StringVar()
        self.valuesdata2 = StringVar()
        self.valuesdata3 = StringVar()
        self.valuesdata4 = StringVar()
        self.valuesdata5 = StringVar()
        self.valuesdata6 = StringVar()
        self.valuesdata7 = StringVar()
        self.textweight = StringVar()
        self.checkbox_train = IntVar()
        self.checkbox_test = IntVar()
        self.value=0
        self.head=StringVar()
        self.data=[]
        self.layer=[]
        self.flag=0
        OPTIONSact = ["Select",
                   "relu",
                   "sigmoid",
                   "tanh"
                ]
        self.variable=StringVar(master)
        self.variable.set(OPTIONSact[0])
        OPTIONSpool = ["Select",
                   "MaxPooling",
                   "AveragePooling"
                ]
        self.layercount=0
        self.datacount=0
        self.variable1=StringVar(master)
        self.variable1.set(OPTIONSpool[0])
        self.yaxis=IntVar()
        self.row=IntVar()
        self.col=IntVar()
        self.band=IntVar()
        self.dtypevalue=IntVar()
        self.epochs=IntVar()
        self.cls=IntVar()
        self.threshold=float()
        self.imgr=StringVar()
        self.imgread=StringVar()
        self.headerr=StringVar()
        self.yaxis=100       
       
        self.master = master
        self.master.title(" Image Classification tool ")
        self.master.config(bg="#dbdbdb")
        self.master.geometry("800x650")
        self.can = Canvas(master, width=785, height=85, bg='#20232A')        
        self.can.place(x=5, y=5)
        self.l1 = ttk.Label(master, text="Recurrent Neural Network Algorithm",font=('times new roman', 18, 'bold'),style='green/black.TLabel',background='#20232A',foreground="#dbdbdb")
        self.l1.place(x=180, y=30)
        self.can = Canvas(master, width=785, height=55, bg='#20232A')        
        self.can.place(x=5, y=90)
        
        self.rb22 = ttk.Checkbutton(master, text='Testing', variable=self.checkbox_test, style='s.TCheckbutton')
        self.rb22.place(x=525, y=110, width=70)
        self.rb21 = ttk.Checkbutton(master, text='Training', variable=self.checkbox_train,style='s.TCheckbutton')
        self.rb21.place(x=35, y=110, width=70)
        
#         self.rb1 = ttk.Radiobutton(master, text='Training', variable=self.checkbox_train, value=5)
#         self.rb1.place(x=32, y=110)
#         self.rb2 = ttk.Radiobutton(master, text='Testing', variable=self.checkbox_train, value=10)
#         self.rb2.place(x=495, y=110)
        
        self.can = Canvas(master, width=785, height=230, bg='#20232A')        
        self.can.place(x=5, y=145)             
        self.l2 = ttk.Label(master,text="Define the parameters for RNN classifier :",font=('times new roman', 12, 'bold'),style='gb.TLabel',background='#20232A',foreground="#dbdbdb")
        self.l2.place(x=25, y=155, width=350)
        
        self.l3 = ttk.Label(master,textvariable=self.valuesdata1,style='gb.TLabel',font=('times new roman', 11, 'bold'),background='#20232A',foreground="#dbdbdb")
        self.l3.place(x=35, y=200, width=250)                      
        self.l4 = ttk.Label(master,textvariable=self.valuesdata2,style='gb.TLabel',font=('times new roman', 11, 'bold'),background='#20232A',foreground="#dbdbdb")
        self.l4.place(x=35, y=230, width=250)    
        self.ll4 = ttk.Label(master,textvariable=self.valuesdata5,style='gb.TLabel',font=('times new roman', 11, 'bold'),background='#20232A',foreground="#dbdbdb")
        self.ll4.place(x=35, y=260, width=250) 
        self.ll6 = ttk.Label(master,textvariable=self.valuesdata6,style='gb.TLabel',font=('times new roman', 11, 'bold'),background='#20232A',foreground="#dbdbdb")
        self.ll6.place(x=35, y=290, width=250) 
        self.ll6 = ttk.Label(master,textvariable=self.valuesdata7,style='gb.TLabel',font=('times new roman', 11, 'bold'),background='#20232A',foreground="#dbdbdb")
        self.ll6.place(x=35, y=320, width=290)
        self.valuesdata1.set("Enter the Epochs value")
        self.valuesdata2.set("Enter Number Of Training Classes")
        self.valuesdata5.set("Enter the Threshold value")
        self.valuesdata6.set("Select Activation Function")
        self.valuesdata7.set("Select MaxPooling/AveragePooling Layer")
        self.e1 =Entry(master)
        self.e1.place(x=485, y=200, width=80)
        self.e2 =Entry(master)
        self.e2.place(x=485, y=230, width=80)
        self.m3 =Entry(master)
        self.m3.place(x=485, y=260, width=80)
        self.ee3=OptionMenu(master,self.variable,*OPTIONSact)
        self.ee3.place(x=485,y=290,width=80)
        self.el3=OptionMenu(master,self.variable1,*OPTIONSpool)
        self.el3.place(x=485,y=320,width=80)              
        
        self.can = Canvas(master, width=785, height=100, bg='#20232A')        
        self.can.place(x=5, y=370) 
        self.b1 = ttk.Button( text='Select ImageFile', style='green/black.TButton' , command=self.img )
        self.b1.place(x=35, y=390, width=110)
        self.e3 = ttk.Label(textvariable=self.valuesdata3,relief="sunken" )
        self.e3.place(x=35, y=420, width=300)        
        self.b2 = ttk.Button( text='Select HeaderFile', style='green/black.TButton' , command=self.header)
        self.b2.place(x=395, y=390, width=110)
        self.e4 = ttk.Label(master,textvariable=self.valuesdata4,relief="sunken")
        self.e4.place(x=395, y=420, width=300)
        
        self.can = Canvas(master, width=785, height=140, bg='#20232A')        
        self.can.place(x=5, y=455) 
        self.l5 = ttk.Label(master,text="Load the weight for Testing :",font=('times new roman', 12, 'bold'),style='gb.TLabel',background='#20232A',foreground="#dbdbdb")
        self.l5.place(x=35, y=475, width=350)
        self.b3 = ttk.Button( text='select weight file', style='add.TButton',command=self.weight )
        self.b3.place(x=45, y=515, width=110)
        self.e3 = Label(master,textvariable=self.textweight,relief="sunken" )
        self.e3.place(x=45, y=545, width=350)
        
        self.can = Canvas(master, width=785, height=50, bg='#20232A')        
        self.can.place(x=5, y=590) 
        self.b7 = ttk.Button(master, text='Next', style='green/black.TButton',command=self.loadtrainingdata)
        self.b7.place(x=645, y=605, width=110)    
        self.b6 = ttk.Button( text='Exit', style='green/black.TButton',command=self.savedone)
        self.b6.place(x=35, y=605, width=110)
        
    def header(self):
        filename =  filedialog.askopenfilename(initialdir = "/",title = "Select file",filetypes = (("Header file","*.hdr"),("all files","*.*")))
        self.valuesdata4.set(filename)
        self.head=filename
        print(self.head) 
        
               
    def img(self):  
        filename =  filedialog.askopenfilename(initialdir = "/",title = "Select file",filetypes = (("all files","*.*"),("Text file","*.txt")))
        self.valuesdata3.set(filename)
        self.image=filename
        print(self.image)
        self.imgread=self.valuesdata3.get()

    def add(self):
        if len(self.e2.get()) == 0:
            messagebox.showerror("Error", "The Number of Training Classes block is empty")
        else:            
            add =  filedialog.askopenfilename(initialdir = "/",title = "Select file",filetypes = (("all files","*.*"),("Text file","*.txt")))
            r=self.e2.get()
            self.cls=int(r)+1
            self.data.append(add)
            self.T.config(state="normal")
            self.T.delete('1.0', END)
            for ent in self.data:
                self.T.insert(END, ent+'\n')
            self.T.config(state="disabled")
            if len(self.data)>=(self.cls-1):
                self.b3.config(state="disabled")
            elif len(self.data)<(self.cls-1):
                self.b3.config(state="normal")
            self.datacount=self.datacount+1
            print(len(self.data))
            print(self.cls)
     
        
    def remove(self):
        self.data=self.data[:-1]
        self.T.config(state="normal")
        self.T.delete('1.0', END)
        for ent in self.data:
            self.T.insert(END, ent+'\n')
        self.T.config(state="disabled")
        if len(self.data)<self.cls:
            self.b3.config(state="normal")
        self.datacount=self.datacount-1
            
    def main(self,file):
        if file.endswith('.hdr'):
            f= open(file,"r")
        else:
            return None
        i=1
        for line in f.readlines():
            line = str(line.lower())
            line = line.strip().lower()
            if str(line) == "datatype: u16" or (str(line) == "datatype: s16"):
#                 D_type=np.uint16
                datavalue=16
            if str(line) == "datatype: u8":
#                 D_type=np.uint8
                datavalue=8
            i=i+1
        return datavalue

    def headerreader(self,headpath):
        dtypevalue=self.main(headpath)
        i=1
        r=-1
        c=-1
        b=-1
        pat1 = re.compile(r"\brows\b", re.IGNORECASE) # but matches because pattern ignores case
        pat2 = re.compile(r"\bcols\b", re.IGNORECASE) # but matches because pattern ignores case
        pat3 = re.compile(r"\bbands\b", re.IGNORECASE)
        pat4 = re.compile(r"\bbildatatype\b", re.IGNORECASE)# but matches because pattern ignores case
        lines = [] #Declare an empty list named "lines"
        with open (headpath, 'rt') as in_file: # Open file txt for reading of text data.
            for line in in_file:
                if pat1.search(line) != None:
                    lines.append(line) 
                    i=i+1
                    r=line.split(" ",1)[1] 
                    print("this is the number of rows extracted:")
                    print(r)

                elif pat2.search(line) != None:
                    lines.append(line) 
                    i=i+1
                    c=line.split(" ",1)[1] 
                    print("this is the number of columns extracted:")
                    print(c)

                elif pat3.search(line) != None:
                    lines.append(line) 
                    i=i+1
                    b=line.split(" ",1)[1] 
                    print("this is the number of bands extracted:")
                    print(b)
                else:
                    i=i+1

        if( r==-1 or b==-1 or c==-1):
            print("ERROR: header information insuffficient")

        in_file.close()
        rr=int(r)
        cc=int(c)
        bb=int(b)
        print("this is the bil datatpe extracted:")
        print(dtypevalue)
        
        return rr,cc,bb,dtypevalue
    
    def savedone(self):
        self.master.destroy()
        
    def definedataload(self):
        self.window9=ttk.tkinter.Toplevel()
        self.window9.geometry("800x295")        
        self.can = Canvas(self.window9, width=785, height=50, bg='#20232A')        
        self.can.place(x=5, y=5)
        self.l11=Label(self.window9,text="Add the Training data",font=('times new roman', 14, 'bold'),style='green/black.TLabel',background='#20232A',foreground="#dbdbdb")
        self.l11.place(x=50,y=20)        
        self.can = Canvas(self.window9, width=785, height=180, bg='#20232A')        
        self.can.place(x=5, y=55)
        self.l5 = Label(self.window9,text="Load the Training data :",font=('times new roman', 12, 'bold'),style='gb.TLabel',background='#20232A',foreground="#dbdbdb")
        self.l5.place(x=35, y=65, width=350)
        self.b3 = Button(self.window9, text='Add', style='add.TButton',command=self.add )
        self.b3.place(x=45, y=105, width=100)
        self.b4 = Button(self.window9, text='Remove', style='remove.TButton',command=self.remove )
        self.b4.place(x=45, y=135, width=100)
        self.l5 = ttk.Label(self.window9,text="Training data :",font=('times new roman', 12, 'bold'),style='gb.TLabel',background='#20232A',foreground="#dbdbdb")
        self.l5.place(x=450, y=65, width=200)
        self.T =Text(self.window9, height=7, width=53)
        self.T.pack()
        self.T.place(x=352,y=100)
        self.T.config({"background": "#20232A"})
        self.T.config({"foreground":"#dbdbdb"})
        self.T.config({"border": 0})
        self.can = Canvas(self.window9, width=785, height=50, bg='#20232A')        
        self.can.place(x=5, y=235) 
        self.b7 = ttk.Button(self.window9, text='Next', style='green/black.TButton',command=self.loadlayer)
        self.b7.place(x=645, y=250, width=110) 
        
    def addextra(self):
        if len(self.e2.get()) == 0:
            messagebox.showerror("Error", "The Number of Training Classes block is empty")
        else:            
            add =  filedialog.askopenfilename(initialdir = "/",title = "Select file",filetypes = (("all files","*.*"),("Text file","*.txt")))
            r=self.e2.get()
            self.cls=int(r)+1
            self.data.append(add)
            self.T.config(state="normal")
            self.T.delete('1.0', END)
            for ent in self.data:
                self.T.insert(END, ent+'\n')
            self.T.config(state="disabled")
            if len(self.data)>=(self.cls-1):
                self.b3.config(state="disabled")
            elif len(self.data)<(self.cls-1):
                self.b3.config(state="normal")
            self.datacount=self.datacount+1
            print(len(self.data))
            print(self.cls)
     
        
    def removeextra(self):
        self.data=self.data[:-1]
        self.T.config(state="normal")
        self.T.delete('1.0', END)
        for ent in self.data:
            self.T.insert(END, ent+'\n')
        self.T.config(state="disabled")
        if len(self.data)<self.cls:
            self.b3.config(state="normal")
        self.datacount=self.datacount-1
        
    def definedataloadextra(self):
        self.window9=ttk.tkinter.Toplevel()
        self.window9.geometry("800x295")        
        self.can = Canvas(self.window9, width=785, height=50, bg='#20232A')        
        self.can.place(x=5, y=5)
        self.l11=Label(self.window9,text="Add the Training data",font=('times new roman', 14, 'bold'),style='green/black.TLabel',background='#20232A',foreground="#dbdbdb")
        self.l11.place(x=50,y=20)        
        self.can = Canvas(self.window9, width=785, height=180, bg='#20232A')        
        self.can.place(x=5, y=55)
        self.l5 = Label(self.window9,text="Load the Training data :",font=('times new roman', 12, 'bold'),style='gb.TLabel',background='#20232A',foreground="#dbdbdb")
        self.l5.place(x=35, y=65, width=350)
        self.b3 = Button(self.window9, text='Add', style='add.TButton',command=self.addextra )
        self.b3.place(x=45, y=105, width=100)
        self.b4 = Button(self.window9, text='Remove', style='remove.TButton',command=self.removeextra )
        self.b4.place(x=45, y=135, width=100)
        self.l5 = ttk.Label(self.window9,text="Training data :",font=('times new roman', 12, 'bold'),style='gb.TLabel',background='#20232A',foreground="#dbdbdb")
        self.l5.place(x=450, y=65, width=200)
        self.T =Text(self.window9, height=7, width=53)
        self.T.pack()
        self.T.place(x=352,y=100)
        self.T.config({"background": "#20232A"})
        self.T.config({"foreground":"#dbdbdb"})
        self.T.config({"border": 0})
        self.can = Canvas(self.window9, width=785, height=50, bg='#20232A')        
        self.can.place(x=5, y=235) 
        self.b7 = ttk.Button(self.window9, text='Next' ,style='green/black.TButton',command=self.loadmodelfortrain)
        self.b7.place(x=645, y=250, width=110)
        
    def loadmodelfortrain(self):
        self.window9.destroy()
        rnnfile.trainmodelexistin(self.data,self.e1.get(),self.m3.get(),self.valuesdata4.get(),self.textweight.get(),)
               
    def loadtrainingdata(self):
        if(self.checkbox_test.get() != 1 and self.checkbox_train.get() != 1):
            messagebox.showerror("Error", "Choose Checkbutton for Training or Testing")            
        elif(self.checkbox_test.get() == 1 and self.checkbox_train.get() != 1):
            val1 = self.valuesdata3.get().split("/")
            val2 = self.valuesdata4.get().split("/")
            if len(self.valuesdata3.get()) == 0:
                messagebox.showerror("Error", "Select the Image file")
            elif len(self.valuesdata4.get()) == 0:
                messagebox.showerror("Error", "Select the header file")
            elif ((val1[-1] + ".hdr") != (val2[-1])):
                messagebox.showerror("Error", "Kindly choose correct hdr/image file for the image")
            elif len(self.textweight.get()) == 0:
                messagebox.showerror("Error", "Select the weight file")
            elif not self.textweight.get().endswith(".hdf5"):
                messagebox.showerror("Error"," Select hdf5 File expected for saved weight")
            else:
                rnnfile.modeltest(self.textweight.get(),self.valuesdata4.get(),self.valuesdata3.get())
        else:
            val1 = self.valuesdata3.get().split("/")
            val2 = self.valuesdata4.get().split("/")
            if len(self.e1.get()) == 0:
                messagebox.showerror("Error", "The value of epochs block is empty")
            elif len(self.e2.get()) == 0:
                messagebox.showerror("Error", "The Number of Training Classes block is empty")
            elif len(self.m3.get()) == 0:
                messagebox.showerror("Error", "The Threshold value block is empty")
            elif len(self.valuesdata3.get()) == 0:
                messagebox.showerror("Error", "Select the Image file")
            elif len(self.valuesdata4.get()) == 0:
                messagebox.showerror("Error", "Select the header file")
            elif ((val1[-1] + ".hdr") != (val2[-1])):
                messagebox.showerror("Error", "Kindly choose correct hdr/image file for the image")
            else:
                if(self.checkbox_test.get() == 1 and self.checkbox_train.get() == 1):
                    self.definedataload()                        
                else:
                    if(self.textweight.get() == ""):
                        MsgBox = messagebox.askquestion ('Select an option','Do you have any existing weight file',icon = 'question')
                        if MsgBox=="yes":                                                
                            if len(self.textweight.get()) == 0:
                                messagebox.showerror("Error", "Select the weight file")
                            else:
                                self.definedataloadextra()                         
                        else:
                            self.definedataload()
                    elif(self.textweight.get() != "" ):
                        if not self.textweight.get().endswith(".hdf5"):
                            messagebox.showerror("Error"," Select the weight file with hdf5 extension")
                        else:
                            MsgBox = messagebox.askquestion ('Select an option','Do you want to append data in this weight file',icon = 'question')
                            if MsgBox=="yes":                                                
                                self.definedataloadextra()                         
                            else:
                                self.definedataload()
                    

    def writetofile(self):
        w=self.e1.get()
        print(w)
        r=self.e2.get()
        print(r)
        t=self.m3.get()
        act=self.variable.get()
        pool=str(self.variable1.get())
        poolc="MaxPooling"
        if(pool==poolc):
            self.value=1
        else:
            self.value=2
        print(self.value)
        value=str(self.value)
        self.file = open('load.txt','w')
        end='//null//'
        self.file.write('epoch:'+w+end)
        self.file.write('cls:'+r+end)
        self.file.write('threshold:'+t+end)
        self.file.write('activation:'+act+end)
        self.file.write('pooling:'+pool+end)
        self.file.write('poolingvalue:'+value+end)
        i=self.valuesdata3.get()
        self.file.write('imgpath;'+i+end)
        h=self.valuesdata4.get()
        self.file.write('headerpath:'+h+end) 
        self.row,self.col,self.band,self.dtypevalue=self.headerreader(self.head)
        self.file.write('row:'+str(self.row)+end)
        self.file.write('col:'+str(self.col)+end)
        self.file.write('band:'+str(self.band)+end)
        self.file.write('dtype:'+str(self.dtypevalue)+end)
        self.file.close()
        self.flag=1
        
    def weight(self):
        filename =filedialog.askopenfilename(initialdir = "/",title = "Select file",defaultextension='.txt')
        self.textweight.set(filename)  
        
    def loadlayer(self):
        self.window9.destroy()
        self.window7=ttk.tkinter.Toplevel()
        """whichNN="addLayer"
        self.window7.protocol("WM_DELETE_WINDOW",self.delete)"""
        self.window7.geometry("596x285")
        self.scrollbar1 = Scrollbar(self.window7)
        self.scrollbar1.pack( side = RIGHT, fill=Y )

        self.T2 =Listbox(self.window7,yscrollcommand = self.scrollbar1.set)
        self.can = Canvas(self.window7, width=385, height=50, bg='#20232A')        
        self.can.place(x=5, y=5)
        self.l11=Label(self.window7,text="Add Layers sequentially in the RNN",font=('times new roman', 14, 'bold'),style='green/black.TLabel',background='#20232A',foreground="#dbdbdb")
        self.l11.place(x=50,y=20)
        self.can = Canvas(self.window7, width=385, height=161, bg='#20232A')        
        self.can.place(x=5, y=58.5)
        self.l22=Label(self.window7,text="Convolution Layer",style='gb.TLabel',font=('times new roman', 12, 'bold'),background='#20232A',foreground="#dbdbdb")
        self.l22.place(x=25,y=90,width=150)
        pool=str(self.variable1.get())
        poolc="MaxPooling"
        if(pool==poolc):
            self.value=1
        else:
            self.value=2
        if(self.value is 1):
            self.l33=Label(self.window7,text="MaxPooling Layer",style='gb.TLabel',font=('times new roman', 12, 'bold'),background='#20232A',foreground="#dbdbdb")
            self.l33.place(x=25,y=125,width=150)
        else:
            self.l33=Label(self.window7,text="AveragePooling Layer",style='gb.TLabel',font=('times new roman', 12, 'bold'),background='#20232A',foreground="#dbdbdb")
            self.l33.place(x=25,y=125,width=150)
        self.l34=Label(self.window7,text="LSTM Layer",style='gb.TLabel',font=('times new roman', 12, 'bold'),background='#20232A',foreground="#dbdbdb")
        self.l34.place(x=25,y=160,width=150)

        self.b22 = ttk.Button(self.window7, text='Add', style='add.TButton' , command=self.addConv)
        self.b22.place(x=260, y=90, width=50)
        self.b44 = ttk.Button(self.window7, text='Add',  style='add.TButton' , command=self.addPool)
        self.b44.place(x=260, y=125, width=50)
        self.b55 = ttk.Button(self.window7, text='Add', style='add.TButton' , command=self.addLSTM)
        self.b55.place(x=260, y=160, width=50)
        self.b33 = ttk.Button(self.window7, text='Remove',  style='remove.TButton' , command=self.delete)
        self.b33.place(x=160, y=190, width=75)
        
        self.can = Canvas(self.window7, width=176.5, height=50, bg='#20232A')        
        self.can.place(x=393.5, y=5)
        self.l56=Label(self.window7,text="Added Layers",font=('times new roman', 14, 'bold'),style='green/black.TLabel',background='#20232A',foreground="#dbdbdb")
        self.l56.place(x=410,y=20) 
        self.T2.place(x=395,y=60,width=180)
        self.can = Canvas(self.window7, width=566.5, height=50, bg='#20232A')        
        self.can.place(x=5, y=225)
        self.b66= ttk.Button(self.window7, text='Load', style='green/black.TButton',command=self.layerdelt)           
#             self.b66= ttk.Button(self.window7, text='Load', style='green/black.TButton',command=lambda : rnnfile.trainmodel(self.data,self.layer))
        self.b66.place(x=450, y=240, width=100)
        self.scrollbar1.config(command = self.T2.yview)
            
    def layerdelt(self):
        self.window7.destroy()
        if(self.checkbox_train.get() == 1 and self.checkbox_test.get() == 1):
            self.writetofile()
            rnnfile.testtrainmodel(self.data,self.layer)
        elif(self.checkbox_train.get() == 1):
            self.writetofile()
            rnnfile.trainmodel(self.data,self.layer)        

    def addConv(self):
                self.layer.append("Conv")
                self.T2.config(state="normal")
                self.T2.delete('0', END)
                for ent in self.layer:
                    self.T2.insert(END, ent+'\n')
                self.T2.config(state="disabled")
                self.layercount=self.layercount+1
                
    def delete(self):
                
                self.layer=self.layer[:-1]
                self.T2.config(state="normal")
                self.T2.delete('0', END)
                for ent in self.layer:
                    self.T2.insert(END, ent+'\n')
                self.T2.config(state="disabled")
                self.layercount=self.layercount-1
            
    def addPool(self):
                if(self.value is 1):
                    self.layer.append("MaxPooling")
                else:
                    self.layer.append("AveragePooling")
                self.T2.config(state="normal")
                self.T2.delete('0', END)
                for ent in self.layer:
                    self.T2.insert(END, ent+'\n')
                self.T2.config(state="disabled")
                self.layercount=self.layercount+1
                
    def addLSTM(self):
                self.layer.append("LSTM")
                self.T2.config(state="normal")
                self.T2.delete('0', END)
                for ent in self.layer:
                    self.T2.insert(END, ent+'\n')
                self.T2.config(state="disabled")
                self.layercount=self.layercount+1
           
if __name__ == '__main__':
        root = Tk()
        root.resizable(0, 0)
        obj = PARAM(root)
        root.mainloop()

Using TensorFlow backend.


F:/RNN Code/RNN Code/rnn_test/RNN_Experiments/generic_binary_500_500_stacked/19_11_18_stacked_indices.hdr
F:/RNN Code/RNN Code/rnn_test/RNN_Experiments/generic_binary_500_500_stacked/19_11_18_stacked_indices
1
3
2
3
100
2
1
this is the number of bands extracted:
     9

this is the number of rows extracted:
    552

this is the number of columns extracted:
    552

this is the bil datatpe extracted:
8
F:/RNN Code/RNN Code/rnn_test/RNN_Experiments/TRAINING_DATA_N/19_11_18_SAMPLE class 1 
F:/RNN Code/RNN Code/rnn_test/RNN_Experiments/TRAINING_DATA_N/19_11_18_others class 2 
F:/RNN Code/RNN Code/rnn_test/RNN_Experiments/TRAINING_DATA_N/19_11_18_SAMPLE ==> 302
F:/RNN Code/RNN Code/rnn_test/RNN_Experiments/TRAINING_DATA_N/19_11_18_others ==> 268
5130 570
maxi t
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1

570/570 [==============================] - 1s 2ms/step - loss: 8.9986e-05 - acc: 1.0000A: 0s - loss: 9.2818e-05 - acc: 1 - ETA: 0s - loss: 9.0778e-05 - acc: 1.
Epoch 29/100
570/570 [==============================] - 1s 2ms/step - loss: 8.4742e-05 - acc: 1.0000
Epoch 30/100
570/570 [==============================] - 1s 2ms/step - loss: 7.9942e-05 - acc: 1.0000
Epoch 31/100
570/570 [==============================] - 1s 2ms/step - loss: 7.5514e-05 - acc: 1.0000
Epoch 32/100
570/570 [==============================] - 1s 2ms/step - loss: 7.1409e-05 - acc: 1.0000
Epoch 33/100
570/570 [==============================] - 1s 2ms/step - loss: 6.7604e-05 - acc: 1.0000
Epoch 34/100
570/570 [==============================] - 1s 2ms/step - loss: 6.4054e-05 - acc: 1.0000
Epoch 35/100
570/570 [==============================] - 1s 2ms/step - loss: 6.0763e-05 - acc: 1.0000A: 0s - loss: 6.0910e-05 - acc: 1.00
Epoch 36/100
570/570 [==============================] - 1s 2ms/step - loss: 5.7684e-05 - acc: 1.0

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 2 Axes>

<Figure size 640x480 with 2 Axes>